In [1]:
import json
import streamlit as st
import pandas as pd
import requests
import mysql.connector
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [2]:
#Fetch the data from mysql
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="XXXXXX",
  database="phonepay"
)
mycursor = mydb.cursor(buffered=True)

#Aggregated_insurance
list1 = "SELECT * FROM aggregated_insurance"
list2 = "SELECT * FROM aggregated_transaction"
list3 = "SELECT * FROM aggregated_user"
list4 = "SELECT * FROM map_insurance"
list5 = "SELECT * FROM map_transaction"
list6 = "SELECT * FROM map_user"
list7 = "SELECT * FROM top_insurance"
list8 = "SELECT * FROM top_transaction"
list9 = "SELECT * FROM top_user"


mycursor.execute(list1)
mydb.commit()
list1 = mycursor.fetchall()

mycursor.execute(list2)
mydb.commit()
list2 = mycursor.fetchall()

mycursor.execute(list3)
mydb.commit()
list3 = mycursor.fetchall()

mycursor.execute(list4)
mydb.commit()
list4 = mycursor.fetchall()

mycursor.execute(list5)
mydb.commit()
list5 = mycursor.fetchall()

mycursor.execute(list6)
mydb.commit()
list6 = mycursor.fetchall()

mycursor.execute(list7)
mydb.commit()
list7 = mycursor.fetchall()

mycursor.execute(list8)
mydb.commit()
list8 = mycursor.fetchall()

mycursor.execute(list9)
mydb.commit()
list9 = mycursor.fetchall()

Aggregated_Insurance = pd.DataFrame(list1, columns=("States", "Years", "Quarter", "Transaction_type", "Insurance_count", "Insurance_amount"))


Aggregated_transaction = pd.DataFrame(list2, columns=("States", "Years", "Quarter", "Transaction_type", "Transaction_count", "Transaction_amount"))


Aggregated_user = pd.DataFrame(list3, columns=("States", "Years", "Quarter", "Brands", "Counts", "Total_Percentage"))




Map_Insurance = pd.DataFrame(list4, columns=("States", "Years", "Quarter", "District", "Insurance_count", "Insurance_amount"))


Map_transaction = pd.DataFrame(list5, columns=("States", "Years", "Quarter", "District", "Transaction_count", "Transaction_amount"))


Map_user = pd.DataFrame(list6, columns=("States", "Years", "Quarter", "District", "Registered_Users", "AppOpens"))



Top_Insurance = pd.DataFrame(list7, columns=("States", "Years", "Quarter", "District", "Insurance_count", "Insurance_amount"))


Top_transaction = pd.DataFrame(list8, columns=("States", "Years", "Quarter", "District", "Transaction_count", "Transaction_amount"))


Top_user = pd.DataFrame(list9, columns=("States", "Years", "Quarter", "District", "Registered_Users", "AppOpens"))



In [16]:
Aggregated_Insurance ["Quarter"].max()

4

In [13]:
#Visualisation of Yearly data of Insurance count and amount
def Agg_insurance (data, year):
    
    EDA = data[data["Years"] == year]
    EDA.reset_index(drop= True, inplace=True)
    EDA = EDA.groupby("States")[["Insurance_count", "Insurance_amount"]].sum()
    EDA = EDA.reset_index()   

    col1,col2= st.columns(2)

    with col1:

        fig_amount= px.bar(EDA, x="States", y= "Insurance_amount",title= f"{year} TRANSACTION AMOUNT",
                           width=600, height= 650, color_discrete_sequence=px.colors.sequential.Aggrnyl)
        st.plotly_chart(fig_amount)
    with col2:

        fig_count= px.bar(EDA, x="States", y= "Insurance_count",title= f"{year} TRANSACTION COUNT",
                          width=600, height= 650, color_discrete_sequence=px.colors.sequential.Bluered_r)
        st.plotly_chart(fig_count)

    col1,col2= st.columns(2)
    with col1:

        url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
        response= requests.get(url)
        data1= json.loads(response.content)
        states_name_tra= [feature["properties"]["ST_NM"] for feature in data1["features"]]
        states_name_tra.sort()
        

        fig_india_1= px.choropleth(EDA, geojson= data1, locations= "States", featureidkey= "properties.ST_NM",
                                 color= "Insurance_amount", color_continuous_scale= "Sunsetdark",
                                 range_color= (EDA["Insurance_amount"].min(),EDA["Insurance_amount"].max()),
                                 hover_name= "States",title = f"{year} TRANSACTION AMOUNT",
                                 fitbounds= "locations",width =600, height= 600)
        fig_india_1.update_geos(visible =False)
        
        st.plotly_chart(fig_india_1)

    with col2:

        fig_india_2= px.choropleth(EDA, geojson= data1, locations= "States", featureidkey= "properties.ST_NM",
                                 color= "Insurance_count", color_continuous_scale= "Sunsetdark",
                                 range_color= (EDA["Insurance_count"].min(),EDA["Insurance_count"].max()),
                                 hover_name= "States",title = f"{year} TRANSACTION COUNT",
                                 fitbounds= "locations",width =600, height= 600)
        fig_india_2.update_geos(visible =False)
        
        st.plotly_chart(fig_india_2)

    return EDA

def Agg_insurance1(data,quarter):
    EDAI= data[data["Quarter"] == quarter]
    EDAI.reset_index(drop= True, inplace= True)

    aiyqg= EDAI.groupby("States")[["Insurance_count", "Insurance_amount"]].sum()
    aiyqg.reset_index(inplace= True)

    col1,col2= st.columns(2)

    with col1:
        fig_q_amount= px.bar(aiyqg, x= "States", y= "Insurance_amount", 
                            title= f"{EDAI['Years'].min()} AND {quarter} TRANSACTION AMOUNT",width= 600, height=650,
                            color_discrete_sequence=px.colors.sequential.Burg_r)
        st.plotly_chart(fig_q_amount)

    with col2:
        fig_q_count= px.bar(aiyqg, x= "States", y= "Insurance_count", 
                            title= f"{EDAI['Years'].min()} AND {quarter} TRANSACTION COUNT",width= 600, height=650,
                            color_discrete_sequence=px.colors.sequential.Cividis_r)
        st.plotly_chart(fig_q_count)

    col1,col2= st.columns(2)
    with col1:

        url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
        response= requests.get(url)
        data1= json.loads(response.content)
        states_name_tra= [feature["properties"]["ST_NM"] for feature in data1["features"]]
        states_name_tra.sort()

        fig_india_1= px.choropleth(aiyqg, geojson= data1, locations= "States", featureidkey= "properties.ST_NM",
                                 color= "Insurance_amount", color_continuous_scale= "Sunsetdark",
                                 range_color= (aiyqg["Insurance_amount"].min(),aiyqg["Transaction_amount"].max()),
                                 hover_name= "States",title = f"{EDAI['Years'].min()} AND {quarter} TRANSACTION AMOUNT",
                                 fitbounds= "locations",width =600, height= 600)
        fig_india_1.update_geos(visible =False)
        
        st.plotly_chart(fig_india_1)
    with col2:

        fig_india_2= px.choropleth(aiyqg, geojson= data1, locations= "States", featureidkey= "properties.ST_NM",
                                 color= "Insurance_count", color_continuous_scale= "Sunsetdark",
                                 range_color= (aiyqg["Insurance_count"].min(),aiyqg["Insurance_count"].max()),
                                 hover_name= "States",title = f"{EDAI['Years'].min()} AND {quarter} TRANSACTION COUNT",
                                 fitbounds= "locations",width =600, height= 600)
        fig_india_2.update_geos(visible =False)
        
        st.plotly_chart(fig_india_2)
    
    return EDAI


#--------------------------------------------------------------------------------------------------

def Aggre_Transaction_type(data, state):
    df_state= data[data["States"] == state]
    df_state.reset_index(drop= True, inplace= True)

    trans= df_state.groupby("Transaction_type")[["Transaction_count", "Transaction_amount"]].sum()
    trans.reset_index(inplace= True)

    col1,col2= st.columns(2)
    with col1:

        fig_hbar_1= px.bar(trans, x= "Transaction_count", y= "Transaction_type", orientation="h",
                        color_discrete_sequence=px.colors.sequential.Aggrnyl, width= 600, 
                        title= f"{state.upper()} TRANSACTION TYPES AND TRANSACTION COUNT",height= 500)
        st.plotly_chart(fig_hbar_1)

    with col2:

        fig_hbar_2= px.bar(trans, x= "Transaction_amount", y= "Transaction_type", orientation="h",
                        color_discrete_sequence=px.colors.sequential.Greens_r, width= 600,
                        title= f"{state.upper()} TRANSACTION TYPES AND TRANSACTION AMOUNT", height= 500)
        st.plotly_chart(fig_hbar_2)


def Aggre_user_plot_1(data,year):
    EDA1= data[data["Years"] == year]
    EDA1.reset_index(drop= True, inplace= True)
    
    EDA1= pd.DataFrame(EDA1.groupby("Brands")["Transaction_count"].sum())
    EDA1.reset_index(inplace= True)

    fig_line_1= px.bar(EDA1, x="Brands",y= "Transaction_count", title=f"{year} BRANDS AND TRANSACTION COUNT",
                    width=1000,color_discrete_sequence=px.colors.sequential.haline_r)
    st.plotly_chart(fig_line_1)

    return EDA1

def Aggre_user_plot_2(data,quarter):
    EDA2= data[data["Quarter"] == quarter]
    EDA2.reset_index(drop= True, inplace= True)

    fig_pie_1= px.pie(data_frame=EDA2, names= "Brands", values="Counts", hover_data= "Total_Percentage",
                      width=1000,title=f"{quarter} QUARTER TRANSACTION COUNT PERCENTAGE",hole=0.5, color_discrete_sequence= px.colors.sequential.Magenta_r)
    st.plotly_chart(fig_pie_1)

    return EDA2

def Aggre_user_plot_3(data,state):
    EDA3= data[data["States"] == state]
    EDA3.reset_index(drop= True, inplace= True)

    aguqyg= pd.DataFrame(EDA3.groupby("Brands")["Counts"].sum())
    aguqyg.reset_index(inplace= True)

    fig_scatter_1= px.line(aguqyg, x= "Brands", y= "Counts", markers= True,width=1000)
    st.plotly_chart(fig_scatter_1)



In [112]:
Aggregated_transaction

,States,Years,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,Andaman & nicobar,2018,1,Recharge & bill payments,4200,1845307
1,Andaman & nicobar,2018,1,Peer-to-peer payments,1871,12138655
2,Andaman & nicobar,2018,1,Merchant payments,298,452507
3,Andaman & nicobar,2018,1,Financial Services,33,10601
4,Andaman & nicobar,2018,1,Others,256,184690
...,...,...,...,...,...,...
8623,West bengal,2023,4,Merchant payments,354067997,239073842398
8624,West bengal,2023,4,Peer-to-peer payments,317959249,964138540494
8625,West bengal,2023,4,Recharge & bill payments,64920775,45319429210
8626,West bengal,2023,4,Financial Services,480740,459007296


In [142]:
def input_year (Year):
    
    EDA1 = Aggregated_transaction[Aggregated_transaction["Years"] == Year]
    EDA1.reset_index(drop= True, inplace=True)
    EDA1 = EDA1.groupby("States")[["Transaction_count", "Transaction_amount"]].sum()
    EDA1 = EDA1.reset_index()   

    agg_trans_amount = px.bar(EDA1, x = "States", y = "Transaction_amount", title = f"{Year} Transaction Amount", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_trans_amount.show()
    agg_trans_count = px.bar(EDA1, x = "States", y = "Transaction_count", title = f"{Year} Transaction Count", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_trans_count.show()

    top = EDA1.sort_values(by=["Transaction_count"], ascending=0)
    top = top.head(10)
    
    agg_top = px.bar(top, x = "States", y = "Transaction_count", title = f"{Year} Top 10 States", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_top.show()
    
    
    Quarter_Data = Aggregated_transaction[Aggregated_transaction["Years"] == Year]
    Quarter_Data.reset_index(drop= True, inplace=True)
    Quarter_Data = Quarter_Data.groupby("Quarter")[["Transaction_count", "Transaction_amount"]].sum()
    Quarter_Data = Quarter_Data.reset_index()
    agg_amount = px.bar(Quarter_Data, x = "Quarter", y = "Transaction_amount", title = f"{Year} Transaction Amount", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_amount.show()
    agg_count = px.bar(Quarter_Data, x = "Quarter", y = "Transaction_count", title = f"{Year} Transaction Count", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_count.show()


    Data_Type = Aggregated_transaction[Aggregated_transaction["Years"] == Year]
    Data_Type.reset_index(drop= True, inplace=True)
    Data_Type = Data_Type.groupby("Transaction_type")[["Transaction_count", "Transaction_amount"]].sum()
    Data_Type = Data_Type.reset_index()
    agg_amount = px.bar(Data_Type, x = "Transaction_type", y = "Transaction_amount", title = f"{Year} Transaction Amount", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_amount.show()
    agg_count = px.bar(Data_Type, x = "Transaction_type", y = "Transaction_count", title = f"{Year} Transaction Count", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    agg_count.show()

input_year (2021)

In [141]:
def input_year (Year):
    
    EDA2 = Aggregated_user[Aggregated_user["Years"] == Year]
    EDA2.reset_index(drop= True, inplace=True)
    EDA2 = EDA2.groupby("States")[["Counts", "Total_Percentage"]].sum()
    EDA2 = EDA2.reset_index()   

    user_count = px.bar(EDA2, x = "States", y = "Counts", title = f"{Year} Total Counts", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    user_count.show()
    

    User_type = Aggregated_user[Aggregated_user["Years"] == Year]
    User_type.reset_index(drop= True, inplace=True)
    
    User_type = User_type.groupby("Brands")["Counts"].sum()
    User_type = User_type.reset_index()
    Users = User_type.sort_values(by= ["Counts"],ascending=0)    
    
    Users = px.bar(Users, x = "Brands", y = "Counts", title = f"{Year} Total Count", color_discrete_sequence=px.colors.sequential.Aggrnyl)
    
    Users.show()


 
input_year (2021)

In [8]:
import streamlit as st
import pandas as pd
import json 
import plotly.express as px
import plotly.io as pio
from sqlalchemy import create_engine,text
from pandas.io import sql
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from streamlit_option_menu import option_menu

In [12]:
dsgn = Image.open("logo.jpg")

with st.sidebar:
    st.image(dsgn,use_column_width=True)
    st.title('Phonepe Pulse')
    title=st.selectbox('***Select***', ['Transactions', 'Users', 'Transactions Map','User Map','Top Transactions','Top Users'])
    Year = st.selectbox(
        '***Select an option from Year***',
        ('2018', '2019', '2020','2021','2022')
    )
    Quarter = st.selectbox(
        '***Select an option from  Quarter***',
        ('1', '2', '3','4')
    )

st.write('You selected:',Year, 'from Year and', Quarter, 'from Quarter.')


fitter_val =st.selectbox('**Select**', ['Transaction_count', 'Transaction_amount','Pincode','Registered_users','Registered_user','App_opens','Count','Amount','Brands','Count','Percentage'])
st.write('**You selected**:',fitter_val )


2024-06-15 09:38:30.760 
  command:

    streamlit run C:\Users\VETRI\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [ ]:

def scatterplot(mysql_df):
        if fitter_val=='Transaction_type':
            fin_filter_val='Transaction_amount'
        if fitter_val=="Amount":
            fin_filter_val='Amount'
        data = [dict(
            type = 'scatter',
            x = mysql_df['State'],
            y = mysql_df[fin_filter_val],
            mode = 'markers',
            transforms = [dict(
                type = 'groupby',
                groups = mysql_df['State'],
               )]
        )]

        fig_dict = dict(data=data)
        pio.show(fig_dict, validate=False)
        st.plotly_chart(fig_dict)
        
        
mytitle3 = st.selectbox('***Select this if scatter Plot***', mysql_df["State"].unique())
if sub_botton:
    if plots=='Line Graph':
        fig = px.line(mysql_df, x='State', y=fin_filter_val)
        fig.show()
        
        
if sub_botton:
    if plots=='Scatter Plot':
                mysql_df = mysql_df.loc[mysql_df['State'] == mytitle3] 
            
           
#                 mysql_df = mysql_df.query("State" == mytitle3)
                scatterplot(mysql_df)
# mysql_df = mysql_df.loc[mysql_df['State'] == mytitle3]            
 
# df_grouped = mysql_df.groupby('state')['amount'].sum().reset_index() fig = px.scatter(df_grouped, x='state', y='amount')
# #check for caps 'State' st.plotly_chart(fig)

In [ ]:
def map_user_plot_1(data, year):
    EDA6= data[data["Years"] == year]
    EDA6.reset_index(drop= True, inplace= True)
    EDA6= EDA6.groupby("States")[["Registered_Users", "AppOpens"]].sum()
    EDA6.reset_index(inplace= True)

    fig_map_user_plot_1= px.line(EDA6, x= "States", y= ["Registered_Users","AppOpens"], markers= True,
                                width=1000,height=800,title= f"{year} REGISTERED USER AND APPOPENS", color_discrete_sequence= px.colors.sequential.Viridis_r)
    st.plotly_chart(fig_map_user_plot_1)

    return EDA6

def map_user_plot_2(data, quarter):
    EDA7= data[data["Quarter"] == quarter]
    EDA7.reset_index(drop= True, inplace= True)
    muyqg= EDA7.groupby("States")[["Registered_Users", "AppOpens"]].sum()
    muyqg.reset_index(inplace= True)

    fig_map_user_plot_1= px.line(muyqg, x= "States", y= ["Registered_Users","AppOpens"], markers= True,
                                title= f"{EDA7['Years'].min()}, {quarter} QUARTER REGISTERED USER AND APPOPENS",
                                width= 1000,height=800,color_discrete_sequence= px.colors.sequential.Rainbow_r)
    st.plotly_chart(fig_map_user_plot_1)

    return EDA7

def map_user_plot_3(data, state):
    EDA8= data[data["States"] == state]
    EDA8.reset_index(drop= True, inplace= True)
    EDA8= EDA8.groupby("District")[["Registered_Users", "AppOpens"]].sum()
    EDA8.reset_index(inplace= True)

    col1,col2= st.columns(2)
    with col1:
        fig_map_user_plot_1= px.bar(EDA8, x= "Registered_Users",y= "District",orientation="h",
                                    title= f"{state.upper()} REGISTERED USER",height=800,
                                    color_discrete_sequence= px.colors.sequential.Rainbow_r)
        st.plotly_chart(fig_map_user_plot_1)

    with col2:
        fig_map_user_plot_2= px.bar(EDA8, x= "AppOpens", y= "District",orientation="h",
                                    title= f"{state.upper()} APPOPENS",height=800,
                                    color_discrete_sequence= px.colors.sequential.Rainbow)
        st.plotly_chart(fig_map_user_plot_2)


In [ ]:
def top_user_plot_1(data,year):
    EDA9= data[data["Years"] == year]
    EDA9.reset_index(drop= True, inplace= True)

    EDA91= pd.DataFrame(EDA9.groupby(["States"])["Registered_Users"].sum())
    EDA91.reset_index(inplace= True)

    fig_top_plot_1= px.bar(EDA91, x= "States", y= "Registered_Users", barmode= "group", color= "Quarter",
                            width=1000, height= 800, color_continuous_scale= px.colors.sequential.Burgyl)
    st.plotly_chart(fig_top_plot_1)

    return EDA9

def top_user_plot_2(data,state):
    EDA10= data[data["States"] == state]
    EDA10.reset_index(drop= True, inplace= True)

    tuysg= pd.DataFrame(EDA10.groupby("Quarter")["Registered_Users"].sum())
    tuysg.reset_index(inplace= True)

    fig_top_plot_1= px.bar(EDA10, x= "Quarter", y= "Registered_Users",barmode= "group",
                           width=1000, height= 800,color= "Registered_Users",hover_data="AppOpens",
                            color_continuous_scale= px.colors.sequential.Magenta)
    st.plotly_chart(fig_top_plot_1)


In [ ]:
Top_Insurance = pd.DataFrame(list7, columns=("States", "Years", "Quarter", "District", "Insurance_count", "Insurance_amount"))


Top_transaction = pd.DataFrame(list8, columns=("States", "Years", "Quarter", "District", "Transaction_count", "Transaction_amount"))


Top_user = pd.DataFrame(list9, columns=("States", "Years", "Quarter", "District", "Registered_Users", "AppOpens"))
